In [1]:
from sklearn.datasets import load_iris
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import numpy as np
import torch.nn.functional as f

In [2]:
# loading the data
data=load_iris()

In [3]:
# shape of the input features (shape of the independent variables)
data.data.shape

(150, 4)

In [4]:
print(data.target_names)
print(data.feature_names)

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
data.data[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [7]:
data.data[:2]
# converting the data into a dataframe
train_data=pd.DataFrame(data.data,columns=data.feature_names)

In [8]:
train_data.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [9]:
# converting the target variable into a dataframe
y_data=pd.DataFrame(data.target,columns=['species'])
y_data.head(3)

,species
0,0
1,0
2,0


In [10]:
# concatinating both the target and the input features into a single dataframe
train_data[['species']]=y_data[['species']]

In [11]:
train_data.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


(150, 5)

In [13]:
# renaming some columns
train_data=train_data.rename(columns={"sepal length (cm)":"sepal_length","sepal width (cm)":"sepal_width",
                   "petal length (cm)":"petal_length","petal width (cm)":"petal_width"})
train_data.shape
train_data.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [15]:
# creating a series which will be used to show how to select certain rows
s = pd.Series(range(-3, 4))
s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [16]:
# selecting rows greater than 1
s[s>1]

5    2
6    3
dtype: int64

In [58]:
s=pd.DataFrame(s,columns=['A'])

c=s[s['A']>0]
c


,A
4,1
5,2
6,3


,A
5,2
6,3


In [19]:
# turning the data into a binary classification problem by dropping the virginica 
#(where does under species=2) rows from the data
train_data=train_data[train_data['species'] <2]

In [20]:
train_data.shape

(100, 5)

In [21]:
train_data.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [22]:
x_data=train_data[['sepal_length','sepal_width','petal_length','petal_width']]
y_data=train_data[['species']]
print('shape of the input data',x_data.shape)
print('shape of the target variable',y_data.shape)


shape of the input data (100, 4)
shape of the target variable (100, 1)


In [23]:
x_data=np.array(x_data)
y_data=np.array(y_data)
x_data=torch.from_numpy(x_data)
y_data=torch.from_numpy(y_data)
print('the sheep for the x_data is',x_data.shape)
print('the sheep for the y_data is',y_data.shape)

the sheep for the x_data is torch.Size([100, 4])
the sheep for the y_data is torch.Size([100, 1])


In [26]:
x_data=Variable(x_data)
y_data=Variable(y_data)

In [30]:
x_data.shape[1]

4

In [55]:
class Sigmoid_with_multi_layers(nn.Module):
    def __init__(self,input_dim):
        super(Sigmoid_with_multi_layers,self).__init__()
        # in this case in_features (is fixed)=the number of features from the data and 
        # out_features can be any arbitrary number
        self.linear1=nn.Linear(input_dim,4)
        # the numbers  can be any arbitrary number but in_feactures_of linear2=out_features_of_linear1
        self.linear2=nn.Linear(4,3)
        # the numbers  can be any arbitrary number but in_feactures_of linear3=out_features_of_linear2
        # out_feature=y_variable 
        self.linear3=nn.Linear(3,1)
    def forward(self,x):
        l1=f.sigmoid(self.linear1(x))
        l2=f.sigmoid(self.linear2(l1))
        l3=f.sigmoid(self.linear3(l2))
        return l3

input_dim=x_data.shape[1]
model=Sigmoid_with_multi_layers(input_dim)
loss_function=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.05,momentum=0.5)

In [57]:


for eporch in range(500):
    # predicting y
    y_pred=model(x_data.float())
    # computing the loss
    loss=loss_function(y_pred,y_data.float())
    print('epoch \n',eporch,'loss \n',loss.data)
    #print(' The True y values are',y_data,)
    #print('and the predicted values for y by the model are')
    
    # Zero gradients, perform a backward pass, and update the weights.
    
    # zero the gradients
    optimizer.zero_grad()
    #perform backward-propagation
    loss.backward()
    # All optimizers implement a step() method, that updates the parameters. It can be used as:
    #optimizer.step()
    optimizer.step()
       

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 
 0 loss 
 tensor(0.8365)
epoch 
 1 loss 
 tensor(0.8303)
epoch 
 2 loss 
 tensor(0.8214)
epoch 
 3 loss 
 tensor(0.8117)
epoch 
 4 loss 
 tensor(0.8019)
epoch 
 5 loss 
 tensor(0.7925)
epoch 
 6 loss 
 tensor(0.7838)
epoch 
 7 loss 
 tensor(0.7757)
epoch 
 8 loss 
 tensor(0.7684)
epoch 
 9 loss 
 tensor(0.7616)
epoch 
 10 loss 
 tensor(0.7555)
epoch 
 11 loss 
 tensor(0.7499)
epoch 
 12 loss 
 tensor(0.7448)
epoch 
 13 loss 
 tensor(0.7401)
epoch 
 14 loss 
 tensor(0.7359)
epoch 
 15 loss 
 tensor(0.7321)
epoch 
 16 loss 
 tensor(0.7287)
epoch 
 17 loss 
 tensor(0.7255)
epoch 
 18 loss 
 tensor(0.7227)
epoch 
 19 loss 
 tensor(0.7201)
epoch 
 20 loss 
 tensor(0.7178)
epoch 
 21 loss 
 tensor(0.7157)
epoch 
 22 loss 
 tensor(0.7137)
epoch 
 23 loss 
 tensor(0.7120)
epoch 
 24 loss 
 tensor(0.7104)
epoch 
 25 loss 
 tensor(0.7090)
epoch 
 26 loss 
 tensor(0.7077)
epoch 
 27 loss 
 tensor(0.7065)
epoch 
 28 loss 
 tensor(0.7055)
epoch 
 29 loss 
 tensor(0.7045)
epoch 
 30 loss 
 te

 244 loss 
 tensor(0.6924)
epoch 
 245 loss 
 tensor(0.6924)
epoch 
 246 loss 
 tensor(0.6924)
epoch 
 247 loss 
 tensor(0.6924)
epoch 
 248 loss 
 tensor(0.6924)
epoch 
 249 loss 
 tensor(0.6924)
epoch 
 250 loss 
 tensor(0.6924)
epoch 
 251 loss 
 tensor(0.6923)
epoch 
 252 loss 
 tensor(0.6923)
epoch 
 253 loss 
 tensor(0.6923)
epoch 
 254 loss 
 tensor(0.6923)
epoch 
 255 loss 
 tensor(0.6923)
epoch 
 256 loss 
 tensor(0.6923)
epoch 
 257 loss 
 tensor(0.6923)
epoch 
 258 loss 
 tensor(0.6922)
epoch 
 259 loss 
 tensor(0.6922)
epoch 
 260 loss 
 tensor(0.6922)
epoch 
 261 loss 
 tensor(0.6922)
epoch 
 262 loss 
 tensor(0.6922)
epoch 
 263 loss 
 tensor(0.6922)
epoch 
 264 loss 
 tensor(0.6922)
epoch 
 265 loss 
 tensor(0.6921)
epoch 
 266 loss 
 tensor(0.6921)
epoch 
 267 loss 
 tensor(0.6921)
epoch 
 268 loss 
 tensor(0.6921)
epoch 
 269 loss 
 tensor(0.6921)
epoch 
 270 loss 
 tensor(0.6921)
epoch 
 271 loss 
 tensor(0.6920)
epoch 
 272 loss 
 tensor(0.6920)
epoch 
 273 loss 
 te

 485 loss 
 tensor(0.6867)
epoch 
 486 loss 
 tensor(0.6866)
epoch 
 487 loss 
 tensor(0.6866)
epoch 
 488 loss 
 tensor(0.6865)
epoch 
 489 loss 
 tensor(0.6865)
epoch 
 490 loss 
 tensor(0.6865)
epoch 
 491 loss 
 tensor(0.6864)
epoch 
 492 loss 
 tensor(0.6864)
epoch 
 493 loss 
 tensor(0.6863)
epoch 
 494 loss 
 tensor(0.6863)
epoch 
 495 loss 
 tensor(0.6862)
epoch 
 496 loss 
 tensor(0.6862)
epoch 
 497 loss 
 tensor(0.6861)
epoch 
 498 loss 
 tensor(0.6861)
epoch 
 499 loss 
 tensor(0.6860)
